In [2]:
sc

<SparkContext master=local[*] appName=PySparkShell>

## Take a look at data (5.2)

In [3]:
from pymongo import MongoClient
import pprint

In [4]:
client=MongoClient('localhost')

In [5]:
db=client.packt

In [6]:
redditCollection=db.reddit

In [7]:
pprint.pprint(redditCollection.find_one())

{u'_id': ObjectId('5a355c33293d03c5e1bf3311'),
 u'archived': True,
 u'author': u'David_ungerer',
 u'author_flair_css_class': None,
 u'author_flair_text': None,
 u'body': u'Good rant, stop looking for a mass movement, if one appears fine, but first change yourself.\nWhile at 54 still a work in process, I am a vegetarian (see corporate tainted food ect.) I shop at farmers markets, small shops and co-ops (member owned) I live in a urban space for two of 900 sq ft. I ride a bike and take MTA. I bank at a Federal Credit Union (small, local, member owned. Most importantly, I choose to live simply.\nAs example . . . Go to a corporate drug store (the checkout girl has no health or retirement  and has to stock shelves and sweep floors for minimum wage or less if the corporation could, and is concerned abouting chatting about the community becouse of the boss.) or ride a mile to an owner run drug store (she welcome me by name, asks about my family and work or the community. She provides health a

## Specify schema

In [8]:
import pyspark.sql.types

In [9]:
from pyspark.sql.types import StructField,StructType, FloatType,StringType,IntegerType

In [10]:
schema=StructType([
                StructField('ups',IntegerType()),
                   StructField('created_utc',StringType()),
                  StructField('postLength',IntegerType())])

## Read Mongo collection into Dataframe

In [11]:
pipeline = "[\
{$project:{'ups':1,'created_utc':1,'_id':0,'postLength':{$size:{$split:['$body',' ']}}}}]"

In [12]:
df = spark.read.format("com.mongodb.spark.sql.DefaultSource").option("uri",
"mongodb://127.0.0.1/packt.reddit").option("pipeline",pipeline).schema(schema).load()

In [13]:
df.printSchema()

root
 |-- ups: integer (nullable = true)
 |-- created_utc: string (nullable = true)
 |-- postLength: integer (nullable = true)



## Convert UTC timestamp to hour

In [14]:
from pyspark.sql.functions import udf
import datetime

In [15]:
getHour=udf(lambda x:datetime.datetime.fromtimestamp(int(x)).hour,IntegerType())

In [16]:
dfClean=df.withColumn('hour',getHour('created_utc'))

In [17]:
dfClean.head()

Row(ups=5, created_utc=u'1262304000', postLength=278, hour=0)

## Preparing data for spark.ml (5.3)

In [18]:
from pyspark.ml.feature import VectorAssembler
# We need to convert DataFrame columns into Vectors

In [19]:
assembler=VectorAssembler(
  inputCols=["postLength","hour"], outputCol="features")

In [20]:
vectorDf=assembler.transform(dfClean)

In [21]:
vectorDf.head()

Row(ups=5, created_utc=u'1262304000', postLength=278, hour=0, features=DenseVector([278.0, 0.0]))

## Predicting up votes (5.4)

In [23]:
from pyspark.ml.regression import LinearRegression
# This is our regressor
from pyspark.ml.evaluation import RegressionEvaluator
# Module to evaluate fit

In [24]:
rf=LinearRegression(labelCol="ups", featuresCol="features")
# Create a linear regressor

In [25]:
(trainingData, testData) = vectorDf.randomSplit([0.7, 0.3])

In [ ]:
model = rf.fit(trainingData)

In [28]:
predictions = model.transform(testData)

In [ ]:
evaluator = RegressionEvaluator(labelCol="ups", \
        predictionCol="prediction", metricName="mae")
rmse = evaluator.evaluate(predictions)


In [31]:
print 'Mean absolute error in number of up votes is %.2f' % rmse
# Add in post length

Mean absolute error in number of up votes is 4.75


In [ ]:
stringIndexer = StringIndexer(inputCol="subreddit", outputCol="subredditIndex")
model = stringIndexer.fit(dfClean)
indexed = model.transform(dfClean)

#encoder = OneHotEncoder(inputCol="subredditIndex", outputCol="subredditVec")
#encoded = encoder.transform(indexed)

In [ ]:
indexed.head()

In [ ]:
indexed.select(['subredditIndex','subreddit']).head(10)

## Convert subredditIndex into vector

In [ ]:
assembler=VectorAssembler(
  inputCols=["subredditIndex"], outputCol="subredditIndexV")

In [ ]:
indexed=assembler.transform(indexed)

In [ ]:
indexed.head(5)

In [ ]:
indexer = StringIndexer(inputCol="subreddit", outputCol="subredditI")
indexedFinal = indexer.fit(df).transform(indexed)

In [ ]:
indexedFinal.head()

In [ ]:
indexedFinal.select('subredditI','subreddit').head(15)

## Set >20 =>21

In [ ]:
setIndex=udf(lambda x:int(x) if x<20 else 21,IntegerType())

In [ ]:
indexedFinalCut=indexedFinal.withColumn('indexCut',setIndex('subredditI'))

In [ ]:
indexedFinalCut.head(5)

## --

In [ ]:
indexer = VectorIndexer(inputCol="subredditIndexV", outputCol="subredditIndexLim", maxCategories=20)

indexerModel=indexer.fit(indexed)

indexedFinal=indexerModel.transform(indexed)

In [ ]:
indexerModel.extractParamMap()

In [ ]:
categoricalFeatures = indexerModel.categoryMaps
print("Chose %d categorical features: %s" %
      (len(categoricalFeatures), ", ".join(str(k) for k in categoricalFeatures.keys())))

In [ ]:
indexedFinal.head()

In [ ]:
dd=indexedFinal.select(['subredditIndexLim'])

In [ ]:
indexedFinalCut.head()

## One-hot encode indexcut

In [ ]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer

In [ ]:
encoder = OneHotEncoder(inputCol="indexCut", outputCol="indexCutHot")
encoded = encoder.transform(indexedFinalCut)
encoded.show()

## Assemble into vector

In [ ]:
inputCols=["indexCutHot","postLength","hour"]
inputCols=["postLength",'hour']

In [ ]:
assembler=VectorAssembler(
  inputCols=inputCols, outputCol="featuresFinal")

In [ ]:
ff=assembler.transform(encoded)

In [ ]:
ff.head()

In [ ]:
rf=RandomForestRegressor(labelCol="ups", featuresCol="featuresFinal", numTrees=10)
# Create a random forest regressor

In [ ]:
from pyspark.ml.regression import LinearRegression

In [ ]:
lr = LinearRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8,\
                      labelCol='ups',featuresCol='featuresFinal')

In [ ]:
(trainingData, testData) = ff.randomSplit([0.7, 0.3])

In [ ]:
model = lr.fit(trainingData)

In [ ]:
predictions = model.transform(testData)

In [ ]:
evaluator = RegressionEvaluator(\
            labelCol="ups", predictionCol="prediction", metricName="mae")
rmse = evaluator.evaluate(predictions)

In [ ]:
print 'Mean absolute error in number of up votes is %.2f' % rmse
# Add in post length

In [ ]:
ff.head()

In [ ]:
from pyspark.sql.functions import udf
stringToVector=udf(lambda x: DenseVector([x]), VectorUDT())

In [ ]:
indexed.head()

In [ ]:
final=indexed.withColumn('subredditV',stringToVector(indexed['subreddit']))

In [ ]:
final.head()

In [ ]:
indexer = VectorIndexer(inputCol="subredditV", outputCol="vSubreddit", maxCategories=10)
indexerModel = indexer.fit(final)

In [ ]:
categoricalFeatures = indexerModel.categoryMaps
print("Chose %d categorical features: %s" %
      (len(categoricalFeatures), ", ".join(str(k) for k in categoricalFeatures.keys())))

In [ ]:
indexedData = indexerModel.transform(final)

In [ ]:
dfClean.head()

In [ ]:
assembler=VectorAssembler(
  inputCols=["hour","postLength","subreddit"], outputCol="features")

In [ ]:
vectorDf=assembler.transform(dfClean)

In [ ]:
vectorDf.head()

In [ ]:
rf=RandomForestRegressor(labelCol="ups", featuresCol="features", numTrees=1)

In [ ]:
(trainingData, testData) = vectorDf.randomSplit([0.7, 0.3])

In [ ]:
pipeline = Pipeline(stages=[rf])

In [ ]:
model = pipeline.fit(trainingData)

In [ ]:
predictions = model.transform(testData)

In [ ]:
evaluator = RegressionEvaluator(\
            labelCol="ups", predictionCol="prediction", metricName="mae")
rmse = evaluator.evaluate(predictions)


In [ ]:
print 'Mean absolute error in number of up votes is %.2f' % rmse
# Add in post length

4.78 to beat